Controlflow Documentation: https://controlflow.ai/welcome

In [1]:
import controlflow as cf
from pydantic import BaseModel

from config import Config
# result = cf.run("Write a short poem about artificial intelligence")

config = Config()
# print(result)

In [2]:
agent = cf.Agent(model=config.CF_FLASH2_MODEL)
agent.run("What is 2*2?")

18:56:10.036 | WARNING | langchain_google_genai._function_utils - Key 'additionalProperties' is not supported in schema, ignoring

18:56:10.039 | WARNING | langchain_google_genai._function_utils - Key 'additionalProperties' is not supported in schema, ignoring

Output()

'4'

In [ ]:
class ResearchProposal(BaseModel):
    title: str
    abstract: str
    key_points: list[str]


@cf.flow
def research_proposal_flow():

    # Task 1: Get the research topic from the user
    user_input = cf.Task(
        "Work with the user to choose a research topic",
        interactive=True,
    )
    
    # Task 2: Generate a structured research proposal
    proposal = cf.run(
        "Generate a structured research proposal",
        result_type=ResearchProposal,
        depends_on=[user_input]
    )
    
    return proposal


result = research_proposal_flow()

print(result.model_dump_json(indent=2))